In [ ]:
import numpy as np

In [ ]:
def normalization(scaler_, df_):
    data_array = df_.values
    data_array = data_array.astype(float)
    
    data_array_normalized = scaler_.transform(data_array)
    df_normalized = pd.DataFrame(data_array_normalized, columns = list(df_.columns))
    
    return df_normalized

In [ ]:
def create_sequences(df, window_size, n_overlap):
    n_rows = len(df)
    if n_overlap == 0:
        n_chunks = n_rows // window_size
        return np.array([df.iloc[i * window_size:(i + 1) * window_size].values for i in range(n_chunks)])
    else:
        overlap = window_size - n_overlap 
        sequences = []
        for start in range(0, n_rows - window_size + 1, overlap):
            end = start + window_size
            sequence = df.iloc[start:end].values
            sequences.append(sequence)
        return np.array(sequences)

In [ ]:
def calc_kurtosis(x):
    value = stats.kurtosis(x, fisher = True, bias = False) 
    if pd.isna(value):
        return 0.0
    else:
        return value

In [ ]:
def calc_skewness(x):
    value = stats.skew(x, bias = False) 
    if pd.isna(value):
        return 0.0
    else:
        return value

In [ ]:
def custom_mape(orig, reconstr):
    # Compute MAPE trying to avoid division by 0 (or too small factors)
    y_true, y_pred = np.array(orig), np.array(reconstr)
    y_true_adjusted = np.where(abs(y_true) <= 0.0001, y_true + 0.01, y_true)
    y_pred_adjusted = np.where(abs(y_true) <= 0.0001, y_pred + 0.01, y_pred)
    return np.mean(np.abs((y_true_adjusted - y_pred_adjusted) / y_true_adjusted))

In [ ]:
def compute_failed_requests(rides_not_served, rides_requested, factor, a, b, c):
    failed = (rides_not_served / rides_requested)
    #weight = min(1, (1/factor)*rides_requested)
    weight = min(1, a*np.log(rides_requested+b)+c)
    w_failed = failed * weight
    return failed, w_failed

In [ ]:
def compute_failed_requests_ratio(rides_not_served, rides_requested, rides_accepted, factor, a, b, c):
    failed = (rides_not_served / rides_accepted)
    #weight = min(1, (1/factor)*rides_requested)
    weight = min(1, a*np.log(rides_requested+b)+c)
    w_failed = failed * weight
    return failed, w_failed

In [ ]:
def compute_service_overhead(total_time, expected_total_time):
    service = (total_time - expected_total_time) / expected_total_time
    return service

In [1]:
'''
def compute_cost_distortion(real_price, expected_price):
    cost = abs(real_price - expected_price) / expected_price
    return cost
'''
def compute_cost_distortion(real_price, actual_price):
    try:
        cost = abs(actual_price - real_price) / real_price
    except:
        cost = 0
    return cost

In [ ]:
def compute_thresholds(failed_req, weight = True):
    if weight:
        return np.percentile(failed_req, 95), np.percentile(failed_req, 97), np.percentile(failed_req, 99)
    else:
        return np.percentile(failed_req, 95), np.percentile(failed_req, 97), np.percentile(failed_req, 99)

In [ ]:
def compute_failure(mean_error, threshold, detect_time, confidence):
    for start in range(0, len(mean_error)):
        if start + detect_time < len(mean_error):
            window = mean_error[start:start + detect_time]
        else:
            window = mean_error[start:]
        timestamps_above = np.sum(window > threshold)
        if timestamps_above / detect_time >= confidence:
            i = window.index[0]
            while i < window.index[len(window) - 1]:
                if window[i] > threshold:
                    return i + detect_time
                else:
                    i += 1

In [1]:
def exceed_thresholds(failed_req, threshold_dict, window_size, condition, perc_95, perc_97, perc_99):
    found_95 = False
    found_97 = False
    found_99 = False
    for start in range(0, len(failed_req)):
        if start + window_size < len(failed_req):
            window = failed_req[start:start + window_size]
        else:
            window = failed_req[start:]
        if not found_95:
            perc_95_above = np.sum(window > perc_95)
            if perc_95_above / window_size >= condition:
                i = window.index[0]
                while i < window.index[len(window) - 1] and not found_95:
                    if window[i] > perc_95:
                        found_95 = True
                        threshold_dict['perc_95'] = i + window_size
                    else:
                        i += 1
        if not found_97:
            perc_97_above = np.sum(window > perc_97)
            if perc_97_above / window_size >= condition:
                i = window.index[0]
                while i < window.index[len(window) - 1] and not found_97:
                    if window[i] > perc_97:
                        found_97 = True
                        threshold_dict['perc_97'] = i + window_size
                    else:
                        i += 1
        if not found_99:
            perc_99_above = np.sum(window > perc_99)
            if perc_99_above / window_size >= condition:
                i = window.index[0]
                while i < window.index[len(window) - 1] and not found_99:
                    if window[i] > perc_99:
                        found_99 = True
                        threshold_dict['perc_99'] = i + window_size
                    else:
                        i += 1
    return threshold_dict